# 01-Nature Tutorial- Vibrational structure

#### Uncomment the cell below to pip install the necessary modules if not already installed

#### Note: Works with Qiskit Version 1.4.1

In [1]:
# %pip install qiskit==1.4.1
# %pip install qiskit-nature
# %pip install qiskit-algorithms

#### Restart the kernel after installing any of the missing packages

### Introduction

This notebook is a tutorial for the vibrational structure of a molecule. We will start off by discussing the molecular Hamiltonian and applying the [Born-Oppenheimer approximation](https://en.wikipedia.org/wiki/Born%E2%80%93Oppenheimer_approximation). There is some background knowledge assumed in this notebook, and it is recommended to review the notebook on the Electronic Structure of a molecule as the same theory lays the foundation for understanding the Vibrational Structure.

We will start with the molecular Hamiltonian for a molecule. This consists of the kinetic and potential energy for both the nucleus and the electrons. The kinetic energy terms involve the gradient of the position squared, while the potential energy terms involve the distance between the particles in the denominator.

$$
\mathcal{H} = - \sum_I \frac{\nabla_{R_I}^2}{M_I} - \sum_i \frac{\nabla_{r_i}^2}{m_e} - \sum_I\sum_i  \frac{Z_I e^2}{|R_I-r_i|} + \sum_i \sum_{j>i} \frac{e^2}{|r_i-r_j|} + \sum_I\sum_{J>I} \frac{Z_I Z_J e^2}{|R_I-R_J|}
$$

The nucleus is denoted by the upper case variables ($M_I$, $R_I$, $Z_I$, and $Z_J$) while the electrons are denoted by the lower case variables. Since the nuclei are much heavier than the electrons they do not move on the same time scale, therefore, the behavior of nuclei and electrons can be decoupled. This reduces our equation to the Born-Oppenheimer approximation. 

With this approximation, the molecular wave function is factorized as a product of an electronic part, this is the solution of the electronic Schrödinger equation, and a vibro-rotational part, which is the solution of the nuclear Schrödinger equation in the potential energy surface (PES) generated by sampling the eigenvalues of the electronic Schrödinger equation for different geometries.

The nuclear Schrödinger equation is usually solved in two steps, similar to the electronic counterpart. The basis functions in this case are called modals, and a single-particle basis is obtained either by the harmonic approximation applied to the PES or from a vibrational self-consistent field (VSCF) calculation. Vibrational anharmonic correlations are added a-posteriori with perturbative or variational approaches. The latter include Vibrational Configuration Interaction (VCI) and Vibrational Coupled Cluster (VCC) for highly-accurate anharmonic energies. The main advantage of VCI and VCC over alternative approaches (such as perturbation theories) is that their accuracy can be systematically improved towards the complete basis set limit for a given PES. However, their applicability is limited to small molecules with up to about 10 atoms due to their unfavorable scaling with system size.

In order to address the scaling problem, we look to use quantum algorithms.

The nuclear Schrödinger equation is

$$
\mathcal{H}_{\text{vib}} |\Psi_{n}\rangle = E_{n} |\Psi_{n}\rangle
$$

The term $\mathcal{H}_{\text{vib}}$ is our Hamiltonian operator and takes the form of a reduced Watson Hamiltonian (reduced because we are neglecting the vibro-rotational coupling term).

$$
  \mathcal{H}_\text{vib}(Q_1, \ldots, Q_L) 
    = - \frac{1}{2} \sum_{l=1}^{L} \frac{\partial^2}{\partial Q_l^2} + V(Q_1, \ldots, Q_L)
$$

where $Q_l$ are the harmonic mass-weighted normal coordinates

$\mathcal{H}_{\text{vib}}$ must be mapped to an operator that acts on the states of a given set of $N_q$ qubits in order to calculate its eigenfunctions on quantum hardware. In electronic structure calculations, the mapping is achieved by expressing the non-relativistic electronic Hamiltonian in second quantization, i.e., by projecting it onto the complete set of antisymmetrized occupation number vectors (ONV) generated by a given (finite) set of orbitals. To encode the vibrational Hamiltonian in an analogous second quantization operator, we expand the potential $V(Q_1, ..., Q_L)$ with the $n$-body expansion as follows:

$$
  V(Q_1, \ldots, Q_L) = V_0 + \sum_{l=1}^L V^{[l]}(Q_l) 
    + \sum_{l<m}^L V^{[l,m]}(Q_l, Q_m) + \sum_{l<m<n}^L V^{[l,m,n]}(Q_l, Q_m, Q_n) + \ldots
$$

where $V_o$ is the electronic energy of the reference geometry, the one-mode term $V^{[l]}(Q_l)$ represents the variation of the PES upon change of the $l$-th normal coordinate from the equilibrium position. Similarly, the two-body potential $V^{[l, m]}(Q_l,Q_m)$ represents the change in the exact PES upon a simultaneous displacement along the $l$-th and $m$-th coordinates. Often, including terms up to three-body in the $L$-body expansion is sufficient to obtain an accuracy of about 1 cm$^{-1}$. We highlight that the many-body expansion of the potential operator defining the Watson Hamiltonian contains arbitrarily high coupling terms. This is a crucial difference compared to the non-relativistic electronic-structure Hamiltonian that contains only pairwise interactions.

A flexible second quantization form of the Watson Hamiltonian is obtained within the $n$-mode representation. Let us assume that each mode $l$ is described by a $N_l$ -dimensional basis set $S_l$ defined as follows:

$$
  \mathcal{S}_l = \{ \phi_1^{(l)} (Q_l) , \ldots , \phi_{N_l}^{(l)} (Q_l) \} \, .
$$

The $n$-mode wave function can be expanded in the product basis $\mathcal{S} = \otimes_{i=1}^L \mathcal{S}_i$ as the following CI-like, Continuous Integration-like, expansion:

$$
  |\Psi\rangle = \sum_{k_1=1}^{N_1} \cdots \sum_{k_L=1}^{N_L} C_{k_1,\ldots,k_L} 
    \phi_{k_1}^{(1)}(Q_1) \cdots \phi_{k_L}^{(L)}(Q_L) \, 
$$

The many-body basis functions $\phi_{k_1}^{(1)}(Q_1) \cdots \phi_{k_L}^{(L)}(Q_L)$ are encoded within the $n$-mode second quantization as occupation number vectors (ONVs) as follows:

$$
  \phi_{k_1}(Q_1) \cdots \phi_{k_L}(Q_L)
                      \equiv  |0_1 \cdots 1_{k_1} \cdots 0_{N_1},
                                   0_1 \cdots 1_{k_2} \cdots 0_{N_2}, 
                                   \cdots , 
                                   0_1 \cdots 1_{k_L} \cdots 0_{N_L}\rangle \, .
$$

The ONV defined above is the product of $L$ mode-specific ONVs, each one describing an individual mode. Since each mode is described by one and only one basis function, the occupation of each mode-specific ONV is one. From a theoretical perspective, each mode can be interpreted as a distinguishable quasi-particle (defined as phonons in solid-state physics). Distinguishability arises from the fact that the PES is not invariant by permutation of two modes, also in this case unlike the Coulomb interaction between two equal particles. From this perspective, a molecule can be interpreted as a collection of $L$ indistinguishable particles that interact through the PES operator.

Based on this second quantization representation we introduce a pair of creation and annihilation operators per mode $l$ *and* per basis function $k_l$ defined as:

$$
  \begin{aligned}
    a_{k_l}^\dagger |\cdots, 0_1 \cdots 0_{k_l} \cdots 0_{N_l}, \cdots\rangle 
      &=  | \cdots, 0_1 \cdots 1_{k_l} \cdots 0_{N_l}, \cdots\rangle \\
    a_{k_l}^\dagger | \cdots, 0_1 \cdots 1_{k_l} \cdots 0_{N_l}, \cdots\rangle &=  0 \\
    a_{k_l} | \cdots, 0_1 \cdots 1_{k_l} \cdots 0_{N_l}, \cdots\rangle
     &= | \cdots, 0_1 \cdots 0_{k_l} \cdots 0_{N_l}, \cdots\rangle \\
    a_{k_l} | \cdots, 0_1 \cdots 0_{k_l} \cdots 0_{N_l}, \cdots\rangle &=  0 \\
  \end{aligned}
$$

with

$$
    \begin{aligned}
    \left[a_{k_l}^\dagger, a_{h_m}^\dagger\right] &= 0 \\
    \left[ a_{k_l}, a_{h_m} \right] &= 0 \\
    \left[ a_{k_l}^\dagger, a_{h_m} \right] &= \delta_{l,m} \, , \delta_{k_l,h_m}
    \end{aligned}
$$

The second quantization form is obtained by expressing the potential as

$$
 \begin{aligned}
  \mathcal{H}_\text{vib}^{SQ} =& \sum_{l=1}^L 
    \sum_{k_l,h_l}^{N_l} \langle \phi_{k_l} | T(Q_l) + V^{[l]}(Q_l) | \phi_{h_l} \rangle a_{k_l}^+ a_{h_l} \\
 +& \sum_{l<m}^L \sum_{k_l,h_l}^{N_l} \sum_{k_m,h_m}^{N_m}
    \langle \phi_{k_l} \phi_{k_m} | V^{[l,m]}(Q_l, Q_m) | \phi_{h_l} \phi_{h_m} \rangle 
    a_{k_l}^+ a_{k_m}^+ a_{h_l} a_{h_m} + \cdots
 \end{aligned}
$$

We highlight here the difference between the operators defined above and the electronic structure one. First, the potential contains (in principle) three- and higher-body coupling terms that lead to strings with six (or more) second quantization operators. Moreover, the Hamiltonian conserves the number of particles for each mode, as can be seen from the fact that the number of creation and annihilation operators for a given mode is the same in each term. Nevertheless, different modes are coupled by two- (and higher) body terms containing the second quantization (SQ) operator belonging to different modes $l$ and $m$. For more information on this topic feel free to check out the following [paper](https://arxiv.org/pdf/2003.12578).

**Compute the electronic potential**

By solving the electronic surface energy (ESE) for different nuclear configurations to obtain the PES function $V(Q_1, \ldots, Q_L)$ we can find the electronic potential. Qiskit gives the possibility to approximate the PES with a quartic force field.

$$
V(Q_1, \ldots, Q_L) = \frac{1}{2}  \sum_{ij} k_{ij} Q_i Q_j
                  + \frac{1}{6}  \sum_{ijk} k_{ijk} Q_i Q_j Q_k
                  + \frac{1}{16} \sum_{ijkl} k_{ijkl} Q_i Q_j Q_k Q_l
$$

The advantage of such a form for the PES is that the anharmonic force fields ($k_{ij}$, $k_{ijk}$, $k_{ijkl}$) can be calculated by finite-difference approaches. For methods for which the nuclear energy Hessian can be calculated analytically with response theory-based methods (such as Hartree-Fock (HF) and Density Functional Theory (DFT)), the quartic force field can be calculated by semi-numerical differentiation as:

$$
k_{ijk} = \frac{H_{ij}(+\delta Q_k) - H_{ij}(-\delta Q_k)}{2\delta Q_k}
$$

and

$$
k_{ijkl} = \frac{H_{ij}(+\delta Q_k+\delta Q_l) - H_{ij}(+\delta Q_k-\delta Q_l)
                    -H_{ij}(-\delta Q_k+\delta Q_l) + H_{ij}(-\delta Q_k+\delta Q_l)}
                    {4\delta Q_k \delta Q_l}
$$

Such numerical procedure is implemented, for instance, in the Gaussian suite of programs.

In practice this can be done with Qiskit using the `GaussianForceDriver`.

In [2]:
# This code is from:
# https://qiskit-community.github.io/qiskit-nature/tutorials/02_vibrational_structure.html

In [3]:
from qiskit_nature.second_q.drivers import GaussianForcesDriver

# if you ran Gaussian elsewhere and already have the output file
driver = GaussianForcesDriver(logfile="aux_files/CO2_freq_B3LYP_631g.log")

# if you want to run the Gaussian job from Qiskit
# driver = GaussianForcesDriver(
#                 ['#p B3LYP/6-31g Freq=(Anharm) Int=Ultrafine SCF=VeryTight',
#                  '',
#                  'CO2 geometry optimization B3LYP/6-31g',
#                  '',
#                  '0 1',
#                  'C  -0.848629  2.067624  0.160992',
#                  'O   0.098816  2.655801 -0.159738',
#                  'O  -1.796073  1.479446  0.481721',
#                  '',
#                  ''])


In [4]:
from qiskit_nature.second_q.problems import HarmonicBasis

basis = HarmonicBasis([2, 2, 2, 2])

In [5]:
from qiskit_nature.second_q.problems import VibrationalStructureProblem
from qiskit_nature.second_q.mappers import DirectMapper

vibrational_problem = driver.run(basis=basis)
vibrational_problem.hamiltonian.truncation_order = 2
main_op, aux_ops = vibrational_problem.second_q_ops()

The optional dependency 'sparse' is not installed. Falling back to using 'numpy' arrays instead. Consider installing the 'sparse' package to reduce memory requirements.


### Map to a qubit Hamiltonian

Now that we have an approximation for the potential, we need to write the Hamiltonian in second quantization. To this end, we need to select a modal basis to calculate the one-body integrals $\langle\phi_{k_i}| V(Q_i) | \phi_{h_i} \rangle$, two-body integrals $\langle\phi_{k_i} \phi_{k_j}| V(Q_i,Q_j) | \phi_{h_i} \phi_{h_j} \rangle \ldots$

In the simplest case, the $\phi$ functions are the harmonic oscillator eigenfunctions for each mode. The main advantage of this choice is that the integrals of a PES expressed as a Taylor expansion are easy to calculate with such a basis. A routine for computing these integrals is implemented in Qiskit.

The bosonic operator, $\mathcal{H}_{\text{vib}}^{SQ}$, is then created and must be mapped to a qubit operator. The direct mapping introduced in the first section of this tutorial can be used in Qiskit as follows:

The vibrational operator for the problem now reads as

In [6]:
print(main_op)

Vibrational Operator
number modes=4, number modals=[2, 2, 2, 2], number terms=272
  (1268.0676746875001+0j) * ( +_0_0 -_0_0 )
+ 0j * ( +_0_0 -_0_1 )
+ 0j * ( +_0_1 -_0_0 )
+ (3813.8767834375008+0j) * ( +_0_1 -_0_1 )
+ (705.8633818750002+0j) * ( +_1_0 -_1_0 )
+ (-46.025705898886045+0j) * ( +_1_0 -_1_1 )
+ (-46.025705898886045+0j) * ( +_1_1 -_1_0 )
+ (2120.1145593750007+0j) * ( +_1_1 -_1_1 )
+ (238.31540750000005+0j) * ( +_2_0 -_2_0 )
+ 0j * ( +_2_0 -_2_1 )
+ 0j * ( +_2_1 -_2_0 )
+ (728.9613775000003+0j) * ( +_2_1 -_2_1 )
+ (238.31540750000005+0j) * ( +_3_0 -_3_0 )
+ 0j * ( +_3_0 -_3_1 )
+ 0j * ( +_3_1 -_3_0 )
+ (728.9613775000003+0j) * ( +_3_1 -_3_1 )
+ 0j * ( +_0_0 -_0_0 +_0_0 -_0_0 )
+ 0j * ( +_0_0 -_0_0 +_0_0 -_0_1 )
+ 0j * ( +_0_0 -_0_0 +_0_1 -_0_0 )
+ 0j * ( +_0_0 -_0_0 +_0_1 -_0_1 )
+ (4.942542500000002+0j) * ( +_0_0 -_0_0 +_1_0 -_1_0 )
+ (-88.20174216876333+0j) * ( +_0_0 -_0_0 +_1_0 -_1_1 )
+ (-88.20174216876333+0j) * ( +_0_0 -_0_0 +_1_1 -_1_0 )
+ (14.827627500000007+0j) * ( +_0_

In the previous cell we defined a bosonic transformation to express the Hamiltonian in the harmonic modal basis with 2 modals per mode with the potential truncated at order 2 and the 'direct' boson to qubit mapping. The calculation is then ran as:

In [7]:
qubit_mapper = DirectMapper()
qubit_op = qubit_mapper.map(main_op)

print(qubit_op)

SparsePauliOp(['IIIIIIII', 'IIIIIIIZ', 'IIIIIIZI', 'IIIIIZII', 'IIIIXXII', 'IIIIYYII', 'IIIIZIII', 'IIIZIIII', 'IIZIIIII', 'IZIIIIII', 'ZIIIIIII', 'IIIIIZIZ', 'IIIIXXIZ', 'IIIIYYIZ', 'IIIIZIIZ', 'IIIIIZZI', 'IIIIXXZI', 'IIIIYYZI', 'IIIIZIZI', 'IIIZIIIZ', 'IIIZIIZI', 'IIIZIZII', 'IIIZXXII', 'IIIZYYII', 'IIIZZIII', 'IIZIIIIZ', 'IIZIIIZI', 'IIZIIZII', 'IIZIXXII', 'IIZIYYII', 'IIZIZIII', 'IZIIIIIZ', 'IZIIIIZI', 'IZIIIZII', 'IZIIXXII', 'IZIIYYII', 'IZIIZIII', 'IZIZIIII', 'IZZIIIII', 'ZIIIIIIZ', 'ZIIIIIZI', 'ZIIIIZII', 'ZIIIXXII', 'ZIIIYYII', 'ZIIIZIII', 'ZIIZIIII', 'ZIZIIIII'],
              coeffs=[ 4.85420003e+03+0.j, -6.18564597e+02+0.j, -1.86053067e+03+0.j,
 -3.49485635e+02+0.j, -2.58640489e+01+0.j, -2.58640489e+01+0.j,
 -1.04971911e+03+0.j, -1.11855863e+02+0.j, -3.42575167e+02+0.j,
 -1.11855863e+02+0.j, -3.42575167e+02+0.j,  1.23563563e+00+0.j,
  2.20504355e+01+0.j,  2.20504355e+01+0.j,  3.70690688e+00+0.j,
  3.70690688e+00+0.j,  6.61513066e+01+0.j,  6.61513066e+01+0.j,
  1.11207206e+0

To have different number of modals per mode:

In [8]:
basis = HarmonicBasis([3, 3, 3, 3])

vibrational_problem = driver.run(basis=basis)
vibrational_problem.hamiltonian.truncation_order = 2
main_op, aux_ops = vibrational_problem.second_q_ops()

qubit_mapper = DirectMapper()

qubit_op = qubit_mapper.map(main_op)

print(qubit_op)

The optional dependency 'sparse' is not installed. Falling back to using 'numpy' arrays instead. Consider installing the 'sparse' package to reduce memory requirements.


SparsePauliOp(['IIIIIIIIIIII', 'IIIIIIIIIIIZ', 'IIIIIIIIIXIX', 'IIIIIIIIIYIY', 'IIIIIIIIIIZI', 'IIIIIIIIIZII', 'IIIIIIIIZIII', 'IIIIIIIXXIII', 'IIIIIIIYYIII', 'IIIIIIXIXIII', 'IIIIIIYIYIII', 'IIIIIIIZIIII', 'IIIIIIXXIIII', 'IIIIIIYYIIII', 'IIIIIIZIIIII', 'IIIIIZIIIIII', 'IIIXIXIIIIII', 'IIIYIYIIIIII', 'IIIIZIIIIIII', 'IIIZIIIIIIII', 'IIZIIIIIIIII', 'XIXIIIIIIIII', 'YIYIIIIIIIII', 'IZIIIIIIIIII', 'ZIIIIIIIIIII', 'IIIIIIIIZIIZ', 'IIIIIIIXXIIZ', 'IIIIIIIYYIIZ', 'IIIIIIXIXIIZ', 'IIIIIIYIYIIZ', 'IIIIIIIZIIIZ', 'IIIIIIXXIIIZ', 'IIIIIIYYIIIZ', 'IIIIIIZIIIIZ', 'IIIIIIIIZXIX', 'IIIIIIIIZYIY', 'IIIIIIIXXXIX', 'IIIIIIIYYXIX', 'IIIIIIIXXYIY', 'IIIIIIIYYYIY', 'IIIIIIXIXXIX', 'IIIIIIYIYXIX', 'IIIIIIXIXYIY', 'IIIIIIYIYYIY', 'IIIIIIIZIXIX', 'IIIIIIIZIYIY', 'IIIIIIXXIXIX', 'IIIIIIYYIXIX', 'IIIIIIXXIYIY', 'IIIIIIYYIYIY', 'IIIIIIZIIXIX', 'IIIIIIZIIYIY', 'IIIIIIIIZIZI', 'IIIIIIIXXIZI', 'IIIIIIIYYIZI', 'IIIIIIXIXIZI', 'IIIIIIYIYIZI', 'IIIIIIIZIIZI', 'IIIIIIXXIIZI', 'IIIIIIYYIIZI', 'IIIIIIZIIIZI', 'IIIIIIII

### Solving the `VibrationalStructureProblem`

We will compute the ground state of our problem instance. To learn more about the individual components that go into the `GroundStateSolver`, please refer to the corresponding tutorials.

In [9]:
# for simplicity, we will use the smaller basis again
vibrational_problem = driver.run(basis=HarmonicBasis([2, 2, 2, 2]))
vibrational_problem.hamiltonian.truncation_order = 2


The optional dependency 'sparse' is not installed. Falling back to using 'numpy' arrays instead. Consider installing the 'sparse' package to reduce memory requirements.


In [10]:
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

solver = GroundStateEigensolver(
    qubit_mapper,
    NumPyMinimumEigensolver(filter_criterion=vibrational_problem.get_default_filter_criterion()),
)


In [11]:
result = solver.solve(vibrational_problem)
print(result)

=== GROUND STATE ===
 
* Vibrational ground state energy (cm^-1): 2432.10695403655
The number of occupied modals for each mode is: 
- Mode 0: 1.0
- Mode 1: 1.0
- Mode 2: 1.0
- Mode 3: 1.0


### Conclusion

Congratulations! You've gained an introductory understanding of the vibrational structure problem and programmed a solution to solve for the vibrational structure of a molecule. There was a lot of information covered in this notebook, so feel free to go back and look over it. 